ERROR: Could not find a version that satisfies the requirement networkd3 (from versions: none)
ERROR: No matching distribution found for networkd3
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import numpy as np
import networkd3 as nd3
import rpy2.robjects as robjects


ModuleNotFoundError: No module named 'networkd3'

In [ ]:
library(rmarkdown)
# Libraries
library(tidyverse)
library(viridis)
library(patchwork)
library(hrbrthemes)
library(circlize)

# Load dataset from github
# data_cite <- read.table("/mnt/ibm_lg/yangjoon.kim/excellxgene_tutorial_manuscript/data/neurips2021_cite/s1d1_gex_cite_leiden001.csv", header=TRUE)
data_cite <- read_csv("/mnt/ibm_lg/yangjoon.kim/excellxgene_tutorial_manuscript/data/neurips2021_cite/s1d1_gex_cite_leiden001.csv")
# Package
library(networkD3)

data_cite_long <- data_cite %>%
  rownames_to_column %>%
  gather(key = 'leiden_GEX_001', value = 'value', -rowname) # %>%
  #filter(value > 0)
colnames(data_cite_long) <- c("source", "target", "value")
data_cite_long$target <- paste(data_cite_long$target, " ", sep="")

# From these flows we need to create a node data frame: it lists every entities involved in the flow
nodes <- data.frame(name=c(as.character(data_cite_long$source), as.character(data_cite_long$target)) %>% unique())
 
# With networkD3, connection must be provided using id, not using real name like in the links dataframe.. So we need to reformat it.
data_cite_long$IDsource=match(data_cite_long$source, nodes$name)-1
data_cite_long$IDtarget=match(data_cite_long$target, nodes$name)-1

# prepare colour scale
# ColourScal ='d3.scaleOrdinal() .range(["#FDE725FF","#B4DE2CFF","#6DCD59FF","#35B779FF","#1F9E89FF","#26828EFF","#31688EFF","#3E4A89FF","#482878FF","#440154FF"])'
# Define the color palettes for the source and target nodes
library(RColorBrewer)
sourceColors <- brewer.pal(9, "Set2")
targetColors <- brewer.pal(9, "Set1")

# generate a histogram to see the distribution of "values", which represents the thickness of the links between the source and the target
hist(data_cite_long$value, main = "Histogram of Value", xlab = "Value")

# Filter the Links data frame to only include links with a value >= threshold
threshold = 30
links_filtered <- data_cite_long[data_cite_long$value >= threshold,]


# saveNetwork(sn, "/mnt/ibm_lg/yangjoon.kim/excellxgene_tutorial_manuscript/celltype_annotation_tutorial/figures/Figure_RNA_CITE_multiome/leiden_GEX_ADT_sankey_R.png")
# pdf(file="/mnt/ibm_lg/yangjoon.kim/excellxgene_tutorial_manuscript/celltype_annotation_tutorial/figures/Figure_RNA_CITE_multiome/leiden_GEX_ADT_sankey_R.pdf",
#     width=6, height=10)
# # Make the Network
sn <- sankeyNetwork(Links = links_filtered, Nodes = nodes,
                     Source = "IDsource", Target = "IDtarget",
                     Value = "value", NodeID = "name",
                     sinksRight=TRUE, colourScale=ColourScal, nodeWidth=40, fontSize=13,
                     #nodeColour = list(sourceColors = sourceColors, targetColors = targetColors),
                     nodePadding=20)
# dev.off